In [1]:
#defines

xsprivileges_base_path = 'C:/Users/I078212/workspace/plc/hana/'
xsprivileges_files = [
    xsprivileges_base_path + 'src/PLC/sap/plc_repl/xs/.xsprivileges',
    xsprivileges_base_path + 'src/PLC/sap/plc/doc/.xsprivileges',
    xsprivileges_base_path + 'src/PLC/sap/plc/xs/.xsprivileges'
]

''' src/PLC/sap/plc/doc/.xsprivileges as an example:
{
    "privileges" : [
        { "name" : "API_Documentation_Read", "description": "Read access for the swagger api documentation"}
    ]
}
'''
#print(xsprivileges_files)

xsc_roles_root_path = 'C:/Users/I078212/workspace/plc/hana/src/PLC/sap/'
xsc_roles_files = [
    xsc_roles_root_path + 'plc/authorizations/Addin_Administrator.hdbrole',
    xsc_roles_root_path + 'plc/authorizations/Addin_User.hdbrole',
    xsc_roles_root_path + 'plc/authorizations/Base_Viewer.hdbrole',
    # TODO: and more ...
]
#print(xsc_roles_files)

In [2]:
# parse .xsprivileges which are in json files

import json

in_jsons = []

for f in xsprivileges_files:
    with open(f) as json_data:
        in_jsons.append(json.load(json_data))

#print(in_jsons[0])

In [3]:
# generate scopes, role-templates (basic part, each is associated with a scope)

scopes = []
for data in in_jsons:
    for priviledge in data['privileges']:
        scope = {}
        scope['name'] = '$XSAPPNAME.' + priviledge['name']
        scope['description'] = priviledge['description']
        scopes.append(scope)
#print(scopes)

role_templates = []

# add the most basic role templates, each is associated with one scope
for data in in_jsons:
    for priviledge in data['privileges']:
        role_template = {}
        role_template['name'] = priviledge['name'] + '_RT'
        role_template['scope-references'] = ['$XSAPPNAME.' + priviledge['name']]
        role_templates.append(role_template)


In [23]:
# read .hdbrole files
xsc_roles_txt = []

# prepare xsc_roles_data[]
for i in range(len(xsc_roles_files)):
    with open(xsc_roles_files[i], 'r') as my_file:
        file_data = my_file.read()
        #print(file_data)
        s = file_data.replace('\t', ' ')
        s = s.strip().replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('\n\n', '\n')
        s = s.replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('\n\n', '\n')
        xsc_roles_txt.append(s)

#print(json.dumps(xsc_roles_txt, indent=4))

In [24]:
# parse .hdbrole files
import re
xsc_roles_parsed = {}

''' XSC role file example:
role sap.plc.authorizations::Contributor
    extends role sap.plc.authorizations::Calculation_Version_Viewer, 
    sap.plc.authorizations::Admin_Costing_And_Analysis_Viewer, 
    sap.plc.authorizations::Admin_Finance_Viewer, 
    sap.plc.authorizations::Admin_Global_Settings_Viewer,
    sap.plc.authorizations::Admin_Logistics_Viewer,
    sap.plc.authorizations::Admin_Prices_Viewer
{
    application privilege: sap.plc.xs::Projects_Create_Update;
    application privilege: sap.plc.xs::Calculations_Create_Update;
    application privilege: sap.plc.xs::Calculation_Versions_Create_Update;
    application privilege: sap.plc.xs::Calculation_Versions_Freeze;
    application privilege: sap.plc.xs::Users_Read;
}
'''
# patterns for role definitions
p1 = re.compile('^role (.*) extends(.*)\{')
p2 = re.compile('^role (.*) \{')


# parse a .hdbrole file, return parsed role
def parse_hdbrole(s):
    parsed_role = {}
    parsed_role['name'] = ''
    parsed_role['extends_roles'] = []
    parsed_role['extends_catalog_roles'] = []
    parsed_role['body_items'] = []

    name = ''
    extends = ''

    m = p1.search(s.replace('\n', ' '))
    if m:
        name = m.group(1)
        extends = m.group(2)
    else:
        m = p2.search(s.replace('\n', ' '))
        name = m.group(1)
    if not m:
        raise Exception('.hdbrole file header does not match pattern')

    # for name
    parsed_role['name'] = name;

    # for extends
    if extends != '':
        for e in extends.split('extends'): # consider multiple extends
            e = e.replace(' ', '')
            if e.startswith('catalogrole'):
                e = e.lstrip('catalogrole')
                for s1 in e.split(','):
                    s1 = s1.strip()
                    parsed_role['extends_catalog_roles'].append(s)
            elif e.startswith('role'):
                e = e.lstrip('role')
                for s1 in e.split(','):
                    parsed_role['extends_roles'].append(s1.strip())

    # parse body in {}
    s = s.replace('\n', '|')
    m = re.match('^.*?\{(.*)?\}', s)
    if not m:
        raise Exception('.hdbrole file body does not match pattern')

    # populate parsed_role['body_items']
    comment = ''
    for s1 in m.group(1).split('|'):
        s1 = s1.replace('  ', ' ').replace('  ', ' ').strip()

        if s1 == '':
            pass
        elif s1.startswith('//') or s1.startswith('--'):
            comment = s1.lstrip('//').strip() # saved for later use
        else:
            item = {}
            item['comment'] = comment
            comment = ''
            item['line'] = s1
            parsed_role['body_items'].append(item)


    # parse application_privileges
    for item in parsed_role['body_items']:
        line1 = item['line']
        item['application_privilege'] = ''
        if line1.startswith('application privilege'):
            line1 = line1.lstrip('application privilege').strip().lstrip(':').strip().strip('"')
            line1 = line1.strip().rstrip(';').strip('"')
            item['application_privilege'] = line1

    # here we do not parse other type of privilege, as xs-security.json is only interested in application_privileges

    return parsed_role


for s in xsc_roles_txt:
    parsed_role = parse_hdbrole(s)
    xsc_roles_parsed[parsed_role['name']] = parsed_role

print(json.dumps(xsc_roles_parsed, indent=4))

{
    "sap.plc.authorizations::Addin_Administrator": {
        "name": "sap.plc.authorizations::Addin_Administrator",
        "extends_roles": [
            "sap.plc.authorizations::Addin_User"
        ],
        "extends_catalog_roles": [],
        "body_items": [
            {
                "comment": "",
                "line": "application privilege: \"sap.plc.xs::AddIn_Edit\";",
                "application_privilege": "sap.plc.xs::AddIn_Edit"
            },
            {
                "comment": "",
                "line": "application privilege: \"sap.plc.xs::AddIn_Read_All\";",
                "application_privilege": "sap.plc.xs::AddIn_Read_All"
            }
        ]
    },
    "sap.plc.authorizations::Addin_User": {
        "name": "sap.plc.authorizations::Addin_User",
        "extends_roles": [
            "sap.plc.authorizations::Base_Viewer"
        ],
        "extends_catalog_roles": [],
        "body_items": [
            {
                "comment": "",
          

In [31]:
# expand xsc_roles_parsed[]: also get all the privileges from extended roles

# extend roles with more roles, do not add the duplicated
def roles_extend_no_dup(roles, more):
    if len(roles) == 0:
        roles.extend(more)
        return roles
    for r in more:
        if r not in roles:
            roles.append(r)
    return roles


# get all the body items for a role
def get_role_body_items(parsed_role):
    if len(parsed_role['extends_roles']) == 0:
        return parsed_role['body_items']

    # init with privileges of itself
    body_items = []
    roles_extend_no_dup(body_items, parsed_role['body_items'])

    for extended_name in parsed_role['extends_roles']:
        if extended_name not in xsc_roles_parsed:
            raise Exception(extended_name + ' is not found in parsed roles')
        extended_items = get_role_body_items(xsc_roles_parsed[extended_name])
        roles_extend_no_dup(body_items, extended_items)

    return body_items;

# calculated 'all_body_items' for each role in xsc_roles_parsed[]
for key, role in xsc_roles_parsed.items():
    role['all_body_items'] = get_role_body_items(role)

#print(json.dumps(xsc_roles_parsed, indent=4))

In [34]:
'''add more role_templates[] from xsc_roles_parsed'''

for parsed_role in xsc_roles_parsed:
    



'add more role_templates[] from xsc_roles_parsed'

In [30]:
# write output file
xs_security_file = 'C:/Users/I078212/tmp/xs-security.json'

xs_security = {}
xs_security['xsappname'] = 'HCO_PLC'
xs_security['scopes'] = scopes
xs_security['role-templates'] = role_templates
#print(xs_security)

import json
with open(xs_security_file, 'w') as out_file:
    out_file.write(json.dumps(xs_security, ensure_ascii=True, indent=4))
